# Scrape data from Fortune500.com

In [15]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [16]:
fortune_url = "http://fortune.com/fortune500/"
year = 2015
fortune_suffix = "/list/filtered?sortBy=profits&first500"

page = requests.get(fortune_url + str(year) + fortune_suffix)

soup = BeautifulSoup(page.content, 'html.parser')

In [20]:
soup.prettify()

'<!DOCTYPE html>\n<html>\n <head>\n  <title data-react-helmet="true">\n   Fortune 500 2015\n  </title>\n  <meta charset="utf-8">\n   <meta content="IE=edge" http-equiv="X-UA-Compatible">\n    <meta content="text/html; charset=utf-8" http-equiv="Content-Type">\n     <meta name="Fortune.com">\n      <meta content="width=device-width, initial-scale=1.0" name="viewport">\n       <meta content="Fortune" data-react-helmet="true" property="og:site_name"/>\n       <meta content="en_US" data-react-helmet="true" property="og:locale"/>\n       <meta content="http://fortune.com/fortune500/2015/list/" data-react-helmet="true" property="og:url"/>\n       <meta content="" data-react-helmet="true" property="og:title"/>\n       <meta content="" data-react-helmet="true" property="og:description"/>\n       <meta content="https://fortunedotcom.files.wordpress.com/2015/06/c.jpg" data-react-helmet="true" property="og:image"/>\n       <meta content="859381007513473" data-react-helmet="true" property="fb:app_

In [21]:
list(soup.children)

['html',
 <html><head><title data-react-helmet="true">Fortune 500 2015</title><meta charset="utf-8"><meta content="IE=edge" http-equiv="X-UA-Compatible"><meta content="text/html; charset=utf-8" http-equiv="Content-Type"><meta name="Fortune.com"><meta content="width=device-width, initial-scale=1.0" name="viewport"><meta content="Fortune" data-react-helmet="true" property="og:site_name"/><meta content="en_US" data-react-helmet="true" property="og:locale"/><meta content="http://fortune.com/fortune500/2015/list/" data-react-helmet="true" property="og:url"/><meta content="" data-react-helmet="true" property="og:title"/><meta content="" data-react-helmet="true" property="og:description"/><meta content="https://fortunedotcom.files.wordpress.com/2015/06/c.jpg" data-react-helmet="true" property="og:image"/><meta content="859381007513473" data-react-helmet="true" property="fb:app_id"/><meta content="@fortunemagazine" data-react-helmet="true" name="twitter:site"/><meta content="" data-react-helme

In [26]:
company_titles = soup.findAll("span", {"class": "company-title"})

In [27]:
company_titles

[<span class="column small-5 company-title" data-reactid=".l4ewahj0s0.2.0.5.1:2.0.0.0.0.4.1:$0-company-list-item-Apple.0.1">Apple</span>,
 <span class="column small-5 company-title" data-reactid=".l4ewahj0s0.2.0.5.1:2.0.0.0.0.4.1:$1-company-list-item-Exxon Mobil.0.1">Exxon Mobil</span>,
 <span class="column small-5 company-title" data-reactid=".l4ewahj0s0.2.0.5.1:2.0.0.0.0.4.1:$2-company-list-item-Wells Fargo.0.1">Wells Fargo</span>,
 <span class="column small-5 company-title" data-reactid=".l4ewahj0s0.2.0.5.1:2.0.0.0.0.4.1:$3-company-list-item-Microsoft.0.1">Microsoft</span>,
 <span class="column small-5 company-title" data-reactid=".l4ewahj0s0.2.0.5.1:2.0.0.0.0.4.1:$4-company-list-item-JP Morgan Chase.0.1">JP Morgan Chase</span>,
 <span class="column small-5 company-title" data-reactid=".l4ewahj0s0.2.0.5.1:2.0.0.0.0.4.1:$5-company-list-item-Berkshire Hathaway.0.1">Berkshire Hathaway</span>,
 <span class="column small-5 company-title" data-reactid=".l4ewahj0s0.2.0.5.1:2.0.0.0.0.4.1:$6

In [32]:
company_revenues = soup.findAll("span", {"class": "company-revenue"})

In [33]:
company_revenues

[<span class="column small-5 company-revenue" data-reactid=".l4ewahj0s0.2.0.5.1:2.0.0.0.0.4.1:$0-company-list-item-Apple.0.2" style="padding:0;">39,510</span>,
 <span class="column small-5 company-revenue" data-reactid=".l4ewahj0s0.2.0.5.1:2.0.0.0.0.4.1:$1-company-list-item-Exxon Mobil.0.2" style="padding:0;">32,520</span>,
 <span class="column small-5 company-revenue" data-reactid=".l4ewahj0s0.2.0.5.1:2.0.0.0.0.4.1:$2-company-list-item-Wells Fargo.0.2" style="padding:0;">23,057</span>,
 <span class="column small-5 company-revenue" data-reactid=".l4ewahj0s0.2.0.5.1:2.0.0.0.0.4.1:$3-company-list-item-Microsoft.0.2" style="padding:0;">22,074</span>,
 <span class="column small-5 company-revenue" data-reactid=".l4ewahj0s0.2.0.5.1:2.0.0.0.0.4.1:$4-company-list-item-JP Morgan Chase.0.2" style="padding:0;">21,762</span>,
 <span class="column small-5 company-revenue" data-reactid=".l4ewahj0s0.2.0.5.1:2.0.0.0.0.4.1:$5-company-list-item-Berkshire Hathaway.0.2" style="padding:0;">19,872</span>,
 